In [44]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC 
import pickle

In [45]:
train_set_file = ['extracted_features.csv', #(a file is a fold)
                  'extracted_features.csv',
                  'extracted_features.csv',
                  'extracted_features.csv',
                  'extracted_features.csv'
                 ]
test_set_file = 'extracted_features.csv'

n_classes = 3
n_folds = 5

In [46]:
#read data set
train_set_list = [pd.read_csv(file_path) for file_path in train_set_file]
test_set_list = pd.read_csv('extracted_features.csv')

In [47]:
#separate X and y
X_train = [train_set.iloc[:, 1:].values for train_set in train_set_list]
y_train = [train_set.iloc[:, 0].values for train_set in train_set_list]
X_test = test_set.iloc[:, 1:].values
y_test = test_set.iloc[:, 0].values

In [48]:
# Feature Scaling Object
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

#5-fold cross validation
from sklearn.model_selection import KFold
X = np.array(X_train)
y = np.array(y_train)
kf = KFold(n_splits=5)
kf.get_n_splits(X)
svm_model_list = []
#all_rf_model = []

for fold ,(train_index, val_index) in enumerate(kf.split(X)):
    
    X_train = sc.fit_transform(np.concatenate(X[train_index],axis=0))
    X_val = sc.transform(X[val_index][0])
    y_train = np.concatenate(y[train_index],axis=0)
    y_val = y[val_index][0]
    
    svm = SVC(kernel='rbf')  
    svm.fit(X_train, y_train)
    
    svm_model_list.append(svm)
    

    #rf = RandomForestClassifier(n_estimators = 400, criterion = "gini")
    #rf.fit(X_train, y_train)
    
    #y_pred_svm = svm.predict(X_val)
    #y_pred_rf = rf.predict(X_val)
    
    #save model
    #pickle.dump(svm, open("svm_" + (fold+1) + ".sav" , 'wb'))
    #pickle.dump(rf, open("rf" + (fold+1) + ".sav" , 'wb'))
    
    
print("trainning finish")

trainning finish


In [49]:
X_test = sc.transform(X_test)
y_score_svm_list = []
y_predict_svm_list = []
for model in svm_model_list:
    y_score_svm_list.append(model.decision_function(X_test))
    y_predict_svm_list.append(model.predict(X_test))
print("test finish")

test finish


In [50]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

n_classes = 3

y_score = y_score_svm_list[0] #try to test on only 1 fold

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i] , pos_label=0)
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot of a ROC curve for a specific class
for i in range(n_classes):
    plt.figure()
    plt.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f)' % roc_auc[i])
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

IndexError: too many indices for array